In [ ]:
import numpy as np

from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, Button, TextInput

from bokeh.layouts import widgetbox, row, column

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.colors import RGB

from bokeh.resources import INLINE 

from matplotlib import cm
from matplotlib.colors import to_hex



from time import sleep, time

import os

import urllib
from notebook import notebookapp

from tqdm import tqdm_notebook as tqdm


output_notebook(resources=INLINE)

In [1]:
#### Constantes physiques :
g = 9.81
####


#### Équations du mouvement : 
def x(t, v0, alpha):
    return v0*np.cos(alpha)*t

def y(t, v0, alpha):
    return -1/2 *g*t**2 + v0*np.sin(alpha)*t
####

In [2]:
#### Équations physiques annexes néccessaires, déduites analytiquement des formule de x(t) et y(t) ci-dessus :

# [ déduite de x(t) et y(t) ]
def fleche(v0, alpha):
    return (v0**2/g)*np.sin(2*alpha)

# [ déduite de y(t) ]
def altitude_max(v0, alpha):
    return ((v0*np.sin(alpha))**2)/(2*g)  

# [ déduite de y(t) ]
def t_of_y0(y0, v0, alpha, descending=1):
    
    delta = (v0*np.sin(alpha))**2 - 2*g*y0
    
    # descending=1 <-> la racine la plus grande est choisie
    # (i.e. à cet instant, l'objet passe par y0 en descendant) 
    
    # descending=-1 <-> la racine la plus petite est choisie
    
    # descending=0 <-> cas d'une seule racine double (delta=0)
    # Lorsque l'on sait que l'on s'attend à delta=0, il est bien plus prudent d'écraser la valeur de delta par 0
    # car son calcul numérique ci-dessus peut en soit donner des valeurs très faibles de n'importe quel signe
    
    
    if descending==0:
        delta = 0
    
    if delta >= 0 :
        sign = descending

        return (v0*np.sin(alpha) + sign*np.sqrt(delta))/g
    
    return None #Pas de racines (réelles)
 
####

In [ ]:
####  Pour l'export video

SAVE_FOLDER = 'output_files'
DEFAULT_ANIM_NAME = 'lancer_54deg'
VIDEO_QUALITY = 18 # pour l'export ffmpeg : nombre entre 0 (lossless) et 51 (très comprimé).
                   # Il n'est pas recommandé d'aller en dessous de 15 (gain de qualité minime).



# Attention, la vitesse d'animation visible sur une vidéo exportée est la vitesse réelle physique
# attendue (avec éventuel ralentissement de lecture SPEED). Une animation vue en live dans le
# notebook sera elle en général plus lente même si elle paraît fluide, à cause des temps de calculs..

####






#### Paramètres physiques :
r = 0.04 #rayon du polygone, en mètres
####


#### Paramètres visuels :
FPS = 65 # Augmente simultanément la précision des courbes



X_RANGE = 2 #en mètres
X_WIDTH = 850 #en pixels
ASPECT_RATIO = 1.9 #rapport largeur sur hauteur

COLORS_MAP_NAME = 'inferno' #nom de la colormap, à choisir parmi https://matplotlib.org/users/colormaps.html
COLOR_MAP_START_FAC = .05
COLOR_MAP_END_FAC = .85
# ces deux nombres entre 0 et 1 permettent sélectionner qu'une partie de la colormap choisie

y0 = 2*r
# altitude de 'visualisation', en mètres (où appraît le bout des trajectoires, lorsque les sliders sont manipulés)
####







#### Réglages des bornes et valeur initiales des divers sliders :

# chaque entrée de 'sliders_parameters' doit être une liste à 3 éléments de la forme : 
# [valeur_inférieure, valeur_supérieure, valeur_initiale]
sliders_params = {
    
    'v0' : [2, 6, 3.6],
    'w0' : [-50, 50, 14],
    'alpha_deg' : [1, 89, 54],
    'Npoly' : [2, 12, 5],
    'a_over_r' : [0, 1, 0],    # a_over_r est le rapport de la distance de décalage 'a' du centre de gravité,
                               # sur le rayon 'r' du polygone.
    'SPEED' : [0.1, 1, 1]
}

####






#### Fonctions d'affichages néccessaires pour un fonctionnement aussi sur serveur.

def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    
    base_url = os.environ['EXTERNAL_URL']
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url



def show_document(doc):
    servers = list(notebookapp.list_running_servers())[0]
    if servers['hostname'] == 'localhost':
        show(doc) 
    else:
        show(doc, notebook_url=remote_jupyter_proxy_url)

####




################  MAIN  ################
def modify_doc(doc):
    global X_WIDTH, sliders_params #variables extérieures modifiées dans le main 




    X_WIDTH = int(X_WIDTH)
    Y_WIDTH = int(X_WIDTH/ASPECT_RATIO)
    if Y_WIDTH % 2 == 1:
        Y_WIDTH -= 1 #hauteur paire en pixels requise pour la génération video ffmpeg

    dt = 1/FPS

    v0 = sliders_params['v0'][2]
    w0 = sliders_params['w0'][2]
    alpha_deg = sliders_params['alpha_deg'][2]
    alpha =alpha_deg*np.pi/180
    Npoly = sliders_params['Npoly'][2]
    a_over_r = sliders_params['a_over_r'][2]
    a = a_over_r * r

    SPEED = sliders_params['SPEED'][2]


    t_fin = t_of_y0(y0, v0, alpha) 


    #Pré-calcule une grande liste (avec grand nombre de points) d'un gradient de couleurs, pour pouvoir ensuite
    #plus tard juste en extraire des sous listes de taille Npoly (à chaque fois que Npoly est modifié)
    #sans devoir régénérer la grande liste
    full_colormap_points = 100
    full_colormap = [to_hex(col) for col in 
                     cm.get_cmap(COLORS_MAP_NAME,full_colormap_points)(range(full_colormap_points))]




    def get_data(v0, w0, alpha, Npoly, a, t_fin, SPEED=SPEED, exact_t_fin=False):
        data_dict = {}
        data_dict['t'] = np.arange(0,t_fin + SPEED*dt, SPEED*dt)
        data_dict['x'] = x(data_dict['t'], v0, alpha)
        data_dict['y'] = y(data_dict['t'], v0, alpha)


        # Puisque le dernier élément de l'array des temps np.arange(0,t_fin + SPEED*dt, SPEED*dt)
        # ci-dessus n'est pas à priori égal à t_fin, exact_t_fin=True assure que le polygone
        # reste à y=cte=y0 lorsque les divers sliders sont manipulés (sinon il y a des 'sautillements').
        # [Le fait que le dernier pas de temps ne soit pas exactement de la même durée queles autres
        # ne se voit pas visuellement en statique, et l'option n'est pas activée lors d'une animation.]

        if exact_t_fin:
            data_dict['x'][-1] = x(t_fin, v0, alpha)
            data_dict['y'][-1] = y(t_fin, v0, alpha)

        #Coordoneées des coins du polygone, à partir de celles du centre de gravité
        for i in range(Npoly):
            #anciennes cordonnées, pour un G centrée (a=0) :
            '''
            data_dict['x_point'+str(i)] = data_dict['x'] + r*np.cos(i*(2*np.pi/Npoly) + w0*data_dict['t'])
            data_dict['y_point'+str(i)] = data_dict['y'] + r*np.sin(i*(2*np.pi/Npoly) + w0*data_dict['t'])
            '''


            #nouvelles cordonnées, pour un G décentrée de a vers les x>0 :
            xi0 = r*np.cos(i*(2*np.pi/Npoly)) - a
            yi0 = r*np.sin(i*(2*np.pi/Npoly))

            ri = np.sqrt(xi0**2 + yi0**2)
            thi0 = np.arctan2(yi0, xi0)

            data_dict['x_point'+str(i)] = data_dict['x'] + ri*np.cos(thi0 + w0*data_dict['t'])
            data_dict['y_point'+str(i)] = data_dict['y'] + ri*np.sin(thi0 + w0*data_dict['t'])

            if exact_t_fin:
                data_dict['x_point'+str(i)][-1] = data_dict['x'][-1] + ri*np.cos(thi0 + w0*t_fin)
                data_dict['y_point'+str(i)][-1] = data_dict['y'][-1] + ri*np.sin(thi0 + w0*t_fin)


        return data_dict

    def get_data_poly(data_dict, a, Npoly):
        poly_data_dict = {}

        current_points_x = [ data_dict['x_point'+str(i)][-1] for i in range(Npoly) ]
        current_points_y = [ data_dict['y_point'+str(i)][-1] for i in range(Npoly) ]

        poly_data_dict['x'] = current_points_x
        poly_data_dict['y'] = current_points_y

        poly_data_dict['color'] = get_color_gradient_list(Npoly)


        # Les masses sont représentées par la taille (rayon) des cercles. La façon dont ces masses varient
        # avec le paramètre d'offset (a/r) est seulement qualitative : le barycentre des positions des masses
        # pondéré par leurs tailles ne donne pas exactement le centre de gravité illustré par la croix noire.

        radius_0 = r/5
        max_additional_radius = a/8
        poly_data_dict['radius'] = [ ( radius_0 + max_additional_radius*np.cos(i*(2*np.pi/Npoly)) ) for i in range(Npoly) ]

        # Remarque : il se trouve que cette configuration de masses (liées aux radius affichés) distribuées
        # selon le cosinus de la position angulaire, donne bien physiquement le centre de gravité affiché
        # si et seulement si max_additional_radius = 2*radius_0*a/r, (i.e 2*a/5 pour radius_0 = r/5) mais
        # ce n'est pas ce réglage qui est choisi il donne lieu à des masses (et rayons) négatives
        # lorsqu'on visualise des offsets a/r > 50% ...        


        return poly_data_dict


    def get_data_impact(v0, alpha):
        impact_data_dict = {}

        x_center = fleche(v0 ,alpha)
        impact_data_dict['x'] = [x_center - r/3, x_center + r/3] 
        impact_data_dict['y'] = [0., 0.]
        return impact_data_dict


    def get_data_G(data_dict, Npoly):
        G_data_dict = {}

        G_data_dict['x'] = [ data_dict['x'][-1] ]
        G_data_dict['y'] = [ data_dict['y'][-1] ]
        return G_data_dict






    def get_color_gradient_list(Npoly):
        #global full_colormap

        def get_unif_subset(L, start_fac, end_fac, N):
            """
               Extrait une sous liste de longueur N d’éléments de a liste L régulièrement (ou au mieux)
               espacés, dont le premier élément et le dernier élément correspondent respectivement
               au '(start_fac)-ème quantile' et au '(end_fac)-ème quantile'.

               (start_fac et end_fac sont des flottants entre 0 et 1)
            """
            i_start = round(start_fac*(len(L)-1))
            i_end = round(end_fac*(len(L)-1))
            step = round( (i_end-i_start)/(N-1) )
            best_i_end = i_start + (N-1)*step
            return L[i_start:best_i_end+1:step]

        return get_unif_subset(full_colormap, 0.15, 0.85, Npoly)






    def create_main_fig_and_source(v0, w0, alpha, Npoly, a, t_fin):
        global p, source, source_poly, source_fleche, source_G, source_time



        data_dict = get_data(v0=v0, w0=w0, alpha=alpha, Npoly=Npoly, a=a, t_fin=t_fin, exact_t_fin=True)

        source = ColumnDataSource( data=data_dict )
        source_poly = ColumnDataSource( data=get_data_poly(data_dict, a, Npoly) )
        source_fleche = ColumnDataSource( data=get_data_impact(v0, alpha) )
        source_G = ColumnDataSource( data=get_data_G(data_dict, Npoly) )

        source_time = ColumnDataSource( data={'x':[1.], 'y':[1.], 'text':[f't = {t_fin:.3f} s']} )


        p = figure(x_range=[0,X_RANGE], y_range=[0,X_RANGE/ASPECT_RATIO], plot_width=X_WIDTH,
                   plot_height=Y_WIDTH, x_axis_label='x [m]', y_axis_label='y [m]',
                   tools="", toolbar_location=None)



        #Trajectoire du centre de gravité
        l = p.line(x='x', y='y', source=source, line_width=2.5, color='black', alpha=.85) 

        colors = get_color_gradient_list(Npoly)
        #Trajectoire de chaque coin du polygone 
        for i in range(Npoly):
            p.line(x='x_point'+str(i), y='y_point'+str(i), source=source, line_width=1.5, alpha=.5, color=colors[i])


        #Affichage du polygone en bout de trajectoire.
        p.patch(x='x', y='y', source=source_poly, fill_color='white', fill_alpha=.9, line_color='darkblue') 

        #Affichage des masses (sommets) du polygone en bout de trajectoire
        p.scatter(x='x', y='y', source=source_poly, radius='radius', fill_color='color', fill_alpha=1, line_color=None)

        #Affichage du point d'impact du centre de gravité (flèche)
        p.line(x='x', y='y', source=source_fleche, color='black', line_width=6)

        #Affichage du centre de gravité
        p.scatter(x='x', y='y', source=source_G, color='black', marker='+', size=210*r, line_width=1.5)

        #Affichage du compteur de temps
        p.text(x='x', y='y', text='text', source=source_time, text_align='center', text_color='dimgrey')


    create_main_fig_and_source(v0, w0, alpha, Npoly, a, t_fin)

    v0_slider = Slider(title="V₀ [m/s]", step=.01,
                       start=sliders_params['v0'][0], end=sliders_params['v0'][1], value=sliders_params['v0'][2])

    w0_slider = Slider(title="ω₀ [rd/s]", step=.1,
                       start=sliders_params['w0'][0], end=sliders_params['w0'][1], value=sliders_params['w0'][2])

    alpha_deg_slider = Slider(title="α [°]", step=.01,
                       start=sliders_params['alpha_deg'][0], end=sliders_params['alpha_deg'][1], value=sliders_params['alpha_deg'][2])

    Npoly_slider = Slider(title="Npoly", step=1,
                       start=sliders_params['Npoly'][0], end=sliders_params['Npoly'][1], value=sliders_params['Npoly'][2])

    a_over_r_slider = Slider(title="Offset du centre de gravité", step=.01, format='0 %',
                       start=sliders_params['a_over_r'][0], end=sliders_params['a_over_r'][1], value=sliders_params['a_over_r'][2]) 

    SPEED_slider = Slider(title="Vitesse de lecture", step=.05, format='0 %',
                       start=sliders_params['SPEED'][0], end=sliders_params['SPEED'][1], value=sliders_params['SPEED'][2])



    def update_points(attrname, old, new):               

        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        a_over_r = a_over_r_slider.value
        a = a_over_r * r

        t_fin = t_of_y0(y0, v0, alpha)

        # Cas (en général des faibles alpha et v0) où la trajectoire n'atteint jamais
        # l'altitude de visualisation y0. -> On choisit alors de les visualiser jusqu'à
        # ce qu'elles atteignent leurs flèche
        if t_fin is None:
            t_fin = t_of_y0(altitude_max(v0,alpha), v0, alpha, descending=0)


        data_dict = get_data(v0, w0, alpha, Npoly, a, t_fin, exact_t_fin=True)


        source.data = data_dict
        source_poly.data = get_data_poly(data_dict, a, Npoly)
        source_fleche.data = get_data_impact(v0, alpha)
        source_G.data = get_data_G(data_dict, Npoly)

        time_dict = source_time.data
        time_dict['text'][0] = f't = {t_fin:.3f} s' # Met à jour le timer visuel
        source_time.data = time_dict



    def refresh_Npoly(attrname, old, new):
        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        a_over_r = a_over_r_slider.value
        a = a_over_r * r

        t_fin = t_of_y0(y0, v0, alpha)
        if t_fin is None:
            t_fin = t_of_y0(altitude_max(v0,alpha), v0, alpha, descending=0)

        create_main_fig_and_source(v0, w0, alpha, Npoly, a, t_fin)

        layout.children[-1].children[0] = p


    v0_slider.on_change('value', update_points)
    w0_slider.on_change('value', update_points) 
    alpha_deg_slider.on_change('value', update_points)
    Npoly_slider.on_change('value', refresh_Npoly)
    a_over_r_slider.on_change('value', update_points)



    play_button = Button(label="Play")
    export_button = Button(label="Export video")
    export_name_textbox = TextInput(value= DEFAULT_ANIM_NAME,title="Nom de l'export",width=290)


    def play_animation(export_video=False):

        t = 0
        t_index = 0

        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        a_over_r = a_over_r_slider.value
        a = a_over_r * r

        SPEED = SPEED_slider.value





        
        # Précalcule toute l'animation ; ensuite à chaque passage de la boucle d'animation ci-dessous,
        # seules les données du prochain timestep seront rajoutées dans l'objet de données dynamique "source.data".
        t_fin = t_of_y0(-r, v0, alpha)
        create_main_fig_and_source(v0, w0, alpha, Npoly, a, t_fin)
        layout.children[-1].children[0] = p
        full_anim_data = get_data(v0, w0, alpha, Npoly, a, t_fin, SPEED=SPEED)

        last_t_index = len(full_anim_data['t']) - 1
        
        
        
        if export_video:
            doc.remove_root(layout) #Sans ça, la méthode export_png de Bokeh bug..



        # Initialise le dict displayed_data en lui insérant que les valeurs à t=0 de toutes les données.
        displayed_data = {}
        for key in full_anim_data:
            displayed_data[key] = full_anim_data[key][0:1]       
        source.data = displayed_data
        source_poly.data = get_data_poly(displayed_data, a, Npoly)
        source_G.data = get_data_G(displayed_data, Npoly)


        all_previous_vertices_in_midair = False


        sleep(.2) #Sinon on ne voit pas toujours le début de l'animation


        if export_video:
            export_png(p, filename=os.path.join(SAVE_FOLDER, ANIM_NAME, f'plot_{t_index}.png'))
            print('Created file ' + os.path.join(SAVE_FOLDER, ANIM_NAME, f'plot_{t_index}.png'))

        
        # La boucle d'animation :
        real_t_start = time()
        for t_index in ( tqdm(range(last_t_index)) if export_video else range(last_t_index) ) :
            # Le module tqdm affiche une barre de chargement lors de l'export video seulement.
            # Cette barre de chargement s'interromprera avant les 100% par les conditions de sorties de boucle.


            sleep(dt)
            t += SPEED*dt
            t_index += 1

            for key in displayed_data:
                displayed_data[key] =  full_anim_data[key][:t_index+1]

            source.data = displayed_data
            source_poly.data = get_data_poly(displayed_data, a, Npoly)
            source_G.data = get_data_G(displayed_data, Npoly)

            time_dict = source_time.data
            time_dict['text'][0] = f't = {t:.3f} s' # Met à jour le timer visuel
            source_time.data = time_dict


            if export_video:
                export_png(p, filename=os.path.join(SAVE_FOLDER, ANIM_NAME, f'plot_{t_index}.png'))
                print('Created file ' + os.path.join(SAVE_FOLDER, ANIM_NAME, f'plot_{t_index}.png'))



            #Conditions d'interruption de l'animation (lorsque l'objet ratterit).

            current_poly_ys = np.array( source_poly.data['y'] )
            current_poly_radiuses = np.array( source_poly.data['radius'] )
            if np.any(current_poly_ys <= current_poly_radiuses):
                if all_previous_vertices_in_midair:
                    # Cas 'normal' où tous les points du solide ont été dans l'air (ordonnée > 0)
                    # au moins un instant
                    # --> arrêt de l'animation dès qu'un d'entre eux repasse sous le sol.
                    break
                elif np.all(current_poly_ys <= current_poly_radiuses):
                    # Cas des lancers où l'objet frole le sol et il n'y a aucun instant
                    # où tous les points du solide ont été dans l'air
                    # --> arrêt de l'animation lorsqu'ils sont tous repassés sous le sol.
                    break




            if np.all(current_poly_ys > current_poly_radiuses):
                all_previous_vertices_in_midair = True

        #if not export_video:
        #    print('(actual animation display time)*SPEED (after initial .2 secs sleep) =', (time()-real_t_start)*SPEED, 's')

    play_button.on_click(play_animation)


    def export_action():
        global ANIM_NAME
        ANIM_NAME = export_name_textbox.value


        ## S'assure que le chemin de destination des images existe bien, sinon le crée.
        if not os.path.exists( os.path.join(SAVE_FOLDER, ANIM_NAME) ):
            os.makedirs( os.path.join(SAVE_FOLDER, ANIM_NAME) )
            print('Created path '+os.path.join(SAVE_FOLDER, ANIM_NAME))


        ## Lance le script d'animation avec option export_video.
        play_animation(export_video=True)

        ### Combinaison des pngs en une viéo par ffmpeg :



        ffmpeg_image_filename = os.path.join(SAVE_FOLDER, ANIM_NAME, 'plot_%d.png')
        ffmpeg_video_filename = os.path.join(SAVE_FOLDER, ANIM_NAME) + '.mp4'

        command = (f'ffmpeg -r {FPS} -f image2 -i "{ffmpeg_image_filename}" -vcodec libx264'
                    f' -crf {VIDEO_QUALITY}  -pix_fmt yuv420p "{ffmpeg_video_filename}"')


        print(f'Executing : {command} ...')
        os.system(command)  
        print(f'Created video file {ffmpeg_video_filename}.')

        doc.add_root(layout) 




    export_button.on_click(export_action)





    row1 = row(alpha_deg_slider, v0_slider, w0_slider)
    row2 = row(Npoly_slider, a_over_r_slider, SPEED_slider)

    row_play = row(play_button, export_button)
    row_name = row(export_name_textbox)
    row_fig = row(p) 



    layout = column( row1, row2 , row_play, row_name, row_fig)


    doc.add_root(layout)






show_document(modify_doc)